In [30]:
from pymongo import MongoClient
import pandas as pd
from getpass import getpass
from dotenv import load_dotenv
import os
import plotly.express as px


In [2]:
token = os.getenv('token')

In [3]:
client = MongoClient("localhost:27017")
db = client['Ironhack']
c = db.get_collection("companies")

In [4]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

In [5]:
condition1 = {"category_code":"games_video"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
games = list(c.find(query, projection))
df = pd.DataFrame(games)
df = df.explode('offices')
df.offices.value_counts()

offices
{'city': 'San Francisco'}    39
{'city': 'New York'}         30
{'city': 'Los Angeles'}      11
{'city': ''}                  8
{'city': 'Paris'}             8
                             ..
{'city': 'Boulder'}           1
{'city': 'Dundee'}            1
{'city': 'TromsÃ¸'}           1
{'city': 'Abercynon'}         1
{'city': 'Vancouver'}         1
Name: count, Length: 118, dtype: int64

In [6]:
condition1 = {"category_code":"web"}
condition2 = {"funding_rounds.raised_amount":{"$gt": 1000000}}
query = {"$and":[condition1,condition2]}
projection = {"name":1, "_id":0, "offices.city":1}
web = list(c.find(query, projection))
df = pd.DataFrame(web)
df = df.explode('offices')
df.offices.value_counts()


offices
{'city': 'San Francisco'}    71
{'city': 'New York'}         65
{'city': 'Palo Alto'}        22
{'city': 'Seattle'}          20
{'city': 'London'}           20
                             ..
{'city': 'Waltham'}           1
{'city': 'Pittsburgh'}        1
{'city': 'Burlingame'}        1
{'city': 'Lowell'}            1
{'city': 'Calabassas'}        1
Name: count, Length: 194, dtype: int64

## SAN FRANCISCO was the chosen location to steel a company's venue.

In [22]:
# Getting all ofices in San Francisco
condition1 = {"number_of_employees":{"$lt": 120}}
condition2 = {"number_of_employees":{"$gte": 90}}
condition3 = {"offices.city":"San Francisco"}
query = {"$and":[condition1,condition2,condition3]}

pipeline = [
    {"$match": query},
    {"$unwind": "$offices"},
    {"$match": {"offices.city": "San Francisco"}},
    {"$project": {"_id": 0, "name": 1, "latitude": "$offices.latitude", "longitude": "$offices.longitude", "address": "$offices.address1"}}
]

final_list = list(c.aggregate(pipeline))

In [27]:
possible_venues = pd.DataFrame(final_list)
possible_venues.drop_duplicates(inplace=True)
possible_venues.dropna(inplace=True)
possible_venues

,name,latitude,longitude,address
0,hi5,37.788668,-122.400558,55 Second Street
1,eBuddy,37.787683,-122.410943,555 Post St.
2,Greystripe,37.710677,-122.393230,160 Spear Street Fl 14
3,Zynga,37.765158,-122.404234,365 Vermont St.
6,99designs,37.795531,-122.400598,447 Battery St.
7,LiveRail,37.787183,-122.397759,575 Market St.
8,Clickpass,37.775196,-122.419204,
11,ServePath,37.789803,-122.389757,345 Spear Street
12,GitHub,37.775196,-122.419204,275 Brannan St.
13,Tealeaf,37.791242,-122.397133,45 Fremont St.


In [152]:
"""
def get_coordinates (where):

    url_geocode = f"https://geocode.xyz/{where}?json=1"
    
    try:
        res = requests.get(url_geocode).json()
        return [res["longt"], res["latt"]]
    
    except:
        print(f"Sorry, no matches for {where}")
        time.sleep(8)
        get_coordinates(where)
"""

In [49]:
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="name", hover_data=["address",])
fig.update_layout(mapbox_style="carto-positron",mapbox_zoom=12)
fig.show()